In [1]:
#read data
import numpy as np
import pandas as pd
df = pd.read_csv('data/SDQS 2017-enejunV2.csv')


//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#tokenize documents

from collections import namedtuple
from nltk.stem import  SnowballStemmer
 
stemmer = SnowballStemmer('spanish')
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(df['Asunto']):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))
    
docs_w2v = []

for i, text in enumerate(df['Asunto']):
    words = text.lower().split()
    words_stemm = [stemmer.stem(i) for i in words]
    docs_w2v.append(words_stemm)

In [4]:
#apply doc2vec

from gensim.models import doc2vec,word2vec
model = word2vec.Word2Vec(docs_w2v, size = 100, window = 300, min_count = 10, workers = 4)

Using TensorFlow backend.


In [6]:
#create a vector of average the words.
average_vect =[]

for sen in docs_w2v:
    
    ave = np.zeros(100)
    for word in sen:
        try:
            ave = ave + model[word]/len(sen)
        except:
            pass
    average_vect.append(ave)
    
        

In [40]:
#find distances between tickets
X = np.array(average_vect)
#s = pd.Series(df['Sector'])
#Y = pd.factorize(s)[0]
Y = pd.get_dummies(df['Sector'])


In [ ]:
#replace low freq# run if needed

df_temp = pd.DataFrame(Y)
df_temp2 = df_temp.apply(lambda x: x.mask(x.map(x.value_counts())<4500, 999) if x.name!='0' else x)
Y = df_temp2

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,Y,random_state=0)


In [11]:
#classification
import xgboost as xgb
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['num_class'] = len(pd.factorize(s)[1])

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))



//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	train-merror:0.412723	test-merror:0.425921
[1]	train-merror:0.375083	test-merror:0.393068
[2]	train-merror:0.353765	test-merror:0.376846
[3]	train-merror:0.338996	test-merror:0.36753
[4]	train-merror:0.328529	test-merror:0.358323
Test error using softmax = 0.3583230579531443


In [46]:
## keras deep learning ##
from keras.models import Sequential

model = Sequential()

In [50]:
from keras.layers import Dense, Activation

model.add(Dense(units=64, input_dim=100))
model.add(Activation('relu'))
model.add(Dense(units=16))
model.add(Activation('softmax'))

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [73]:
# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(np.array(X_train), np.array(y_train), epochs=15, batch_size=32)
#model.train_on_batch(x_batch, y_batch)


#Evaluate your performance in one line:
loss_and_metrics = model.evaluate(np.array(X_test), np.array(y_test), batch_size=128)

#Or generate predictions on new data:
classes = model.predict(np.array(X_test), batch_size=128)

Epoch 1/15
109485/109485 [==============================] - 8s - loss: 1.0316 - acc: 0.6954     
Epoch 2/15
109485/109485 [==============================] - 8s - loss: 1.0198 - acc: 0.6995     
Epoch 3/15
109485/109485 [==============================] - 8s - loss: 1.0083 - acc: 0.7032     
Epoch 4/15
109485/109485 [==============================] - 8s - loss: 0.9980 - acc: 0.7044     
Epoch 5/15
109485/109485 [==============================] - 8s - loss: 0.9886 - acc: 0.7067     
Epoch 6/15
109485/109485 [==============================] - 8s - loss: 0.9790 - acc: 0.7084     
Epoch 7/15
109485/109485 [==============================] - 8s - loss: 0.9692 - acc: 0.7108     
Epoch 8/15
109485/109485 [==============================] - 8s - loss: 0.9614 - acc: 0.7129     
Epoch 9/15
109485/109485 [==============================] - 8s - loss: 0.9539 - acc: 0.7148     
Epoch 10/15
109485/109485 [==============================] - 8s - loss: 0.9460 - acc: 0.7154     
Epoch 11/15
109485/109485 [==

In [74]:
sum(np.argmax(classes, axis=1)==np.argmax(np.array(y_test), axis=1))/len(np.argmax(classes, axis=1))

0.70957665433621042

In [2]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [68]:
### keras with iris###
type(np.array(y_test))

pandas.core.frame.DataFrame

In [32]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = pandas.read_csv("iris.csv") #, header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


In [36]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 97.33% (4.42%)


In [86]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [20]:
### wordtovec with subtema.
### NOT WORKING ###

from collections import namedtuple
from nltk.stem import  SnowballStemmer
stemmer = SnowballStemmer('spanish')

#word to vec with subtema
Subtema = np.unique(df['Subtema'])

#replace low freq# run if needed
#df_temp = pd.DataFrame(Subtema)
#df_temp2 = df_temp.apply(lambda x: x.mask(x.map(x.value_counts())<15, 'OTROS') if x.name!='0' else x)
#Y = df_temp2

docs_w2v=[]

for i, text in enumerate(Subtema):
    words = text.lower().split()
    words_stemm = [stemmer.stem(j) for j in words]
    docs_w2v.append(words_stemm)

In [35]:
from gensim.models import doc2vec,word2vec
model = word2vec.Word2Vec(docs_w2v, size = 200, window = 300, min_count = 1, workers = 4)


In [36]:
## biuld the average by row
average_vect =[]
for sen in docs_w2v:
    
    ave = np.zeros(200)
    for word in sen:

        ave = ave + model[word]/len(sen)        
    average_vect.append(ave)

In [38]:
from sklearn.cluster import SpectralClustering
mat = np.matrix(average_vect)
labels = SpectralClustering(100).fit_predict(mat)

In [32]:
import numpy as np
from sklearn.cluster import KMeans

### For the purposes of this example, we store feature data from our
### dataframe `df`, in the `f1` and `f2` arrays. We combine this into
### a feature matrix `X` before entering it into the algorithm.
#f1 = df['Distance_Feature'].values
#f2 = df['Speeding_Feature'].values

X=np.matrix(average_vect)
kmeans = KMeans(n_clusters=100).fit(X)

In [39]:
data = {'subtema': Subtema, 'cluster':labels}
pd.DataFrame(data).to_csv('subtema.csv')